In [1]:
import geopandas as gpd
from myutils import *
import fiona

## Convierte a minúsculas todo el contenido de los archivos geopackage
En la siguiente celda se ejecutan las funciones que convierten todos los valores de las capas a minúsculas, y añade los campos de latitud y longitud en el sistema 4326

In [ ]:
file = r'../GeoData/Polygons.gpkg'
layers = [l for l in fiona.listlayers(file) if 'style' not in l]
print(layers)

# Para cada capa, se aplica la función lower_content y add_latlon
for layer in layers:
    df = gpd.read_file(file, layer = layer)
    lower_content(df, filepath=file, layer=layer)
    add_latlon(df, filepath=file, layer=layer)

### Convierte en datos numéricos los campos necesarios

In [ ]:
file = r'../GeoData/Polygons.gpkg'
layers = ['manzanas']
columns = {'pobtot':int, 'vivtot':int, 'vivpar':int, 'tvipahab':int, 'prom_ocup':float}

print(layers)

# Para cada capa, se aplica la función lower_content y add_latlon
for layer in layers:
    df = gpd.read_file(file, layer = layer)
    column_to_numeric(df, columns=columns, filepath=file, layer=layer)

### junta los archivos del catastro y del SEDUVI de todas las alcaldías

In [2]:
path_origin = r'../GeoData/raw/'


nombres = ['alvaro_obregon', 'azcapotzalco', 'benito_juarez', 'coyoacan', 'cuajimalpa', 'cuauhtemoc', 'gustavo_a_madero', 'iztacalco', 'iztapalapa', 'magdalena_contreras', 'miguel_hidalgo', 'milpa_alta', 'tlahuac', 'tlalpan', 'venustiano_carranza', 'xochimilco']
nombres_b = ['álvaro obregón', 'azcapotzalco', 'benito juárez', 'coyoacán', 'cuajimalpa de morelos', 'cuauhtémoc', 'gustavo a madero', 'iztacalco', 'iztapalapa', 'magdalena contreras', 'miguel hidalgo', 'milpa alta', 'tláhuac', 'tlalpan', 'venustiano carranza', 'xochimilco']
claves = ['09010', '09002','09014','09003','09004','09015','09005','09006','09007','09008','09016','09009','09011','09012','09017','09013']

columns = {'superficie':float, 'niveles':int, 'altura':float, 'area_libre':int, 'minimo_viv':int}

dfs_catastro = []
dfs_seduvi = []


In [3]:
for nombre, nombre_b, clave in zip(nombres, nombres_b, claves):
    print(nombre)
    file_catastro = path_origin + 'catastro/catastro2021_'+ nombre.upper() + '/catastro2021_' + nombre.upper() + '.shp'
    df_catastro = gpd.read_file(file_catastro)
    
    df_catastro['alcaldia'] = nombre_b
    df_catastro['cve_alc'] = clave
    
    
    file_seduvi = path_origin + 'SEDUVI/'+ nombre + '/' + nombre + '.shp'
    df_seduvi = gpd.read_file(file_seduvi)
    
    df_seduvi = df_seduvi.rename(columns={'latitud':'lat', 'longitud':'lon'})
    
    df_seduvi = lower_content(df_seduvi)
    df_seduvi = column_to_numeric(df_seduvi, columns)
    
    df_catastro['alcaldia'] = nombre_b
    df_catastro['cve_alc'] = clave
    
    dfs_catastro.append(df_catastro)
    dfs_seduvi.append(df_seduvi)
    
    print('Completado: ' + nombre_b + ' ' + clave)


alvaro_obregon
Completado: álvaro obregón 09010
azcapotzalco
Completado: azcapotzalco 09002
benito_juarez
Completado: benito juárez 09014
coyoacan
Completado: coyoacán 09003
cuajimalpa
Completado: cuajimalpa de morelos 09004
cuauhtemoc
Completado: cuauhtémoc 09015
gustavo_a_madero
Completado: gustavo a madero 09005
iztacalco
Completado: iztacalco 09006
iztapalapa
Completado: iztapalapa 09007
magdalena_contreras
Completado: magdalena contreras 09008
miguel_hidalgo
Completado: miguel hidalgo 09016
milpa_alta
Completado: milpa alta 09009
tlahuac
Completado: tláhuac 09011
tlalpan
Completado: tlalpan 09012
venustiano_carranza
Completado: venustiano carranza 09017
xochimilco
Completado: xochimilco 09013


In [11]:
for p in dfs_catastro:
    p['fid'] = p['fid'].astype(float)
    p['fid'] = p['fid'].astype(int)

In [12]:

catastro_cdmx = pd.concat(dfs_catastro)
catastro_cdmx = gpd.GeoDataFrame(catastro_cdmx, geometry='geometry')

seduvi_cdmx = pd.concat(dfs_seduvi)
seduvi_cdmx = gpd.GeoDataFrame(seduvi_cdmx, geometry='geometry')

catastro_cdmx.to_file('../GeoData/catastro.gpkg', layer='catastro_cdmx', driver='GPKG')
seduvi_cdmx.to_file('../GeoData/catastro.gpkg', layer='seduvi_cdmx', driver='GPKG')



In [ ]:
nombre = 'gustavo_a_madero'
file_seduvi = path_origin + 'SEDUVI/'+ nombre + '/' + nombre + '.shp'
df_seduvi = gpd.read_file(file_seduvi)

df_seduvi = df_seduvi.rename(columns={'latitud':'lat', 'longitud':'lot'})

df_seduvi = lower_content(df_seduvi)
df_seduvi = column_to_numeric(df_seduvi, columns)
